In [3]:
import torch
import json
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForMultipleChoice
import torch.nn.functional as F
import torch.nn as nn

In [4]:
# Reading the train dataset back
with open('train_dataset_final.json', 'r') as train_file:
    train_dataset_back = json.load(train_file)

# Reading the test dataset back
with open('test_dataset_final.json', 'r') as test_file:
    test_dataset_back = json.load(test_file)

# Display the sizes of the two datasets
print(f"Size of the train dataset: {len(train_dataset_back)}")
print(f"Size of the test dataset: {len(test_dataset_back)}")

Size of the train dataset: 1338
Size of the test dataset: 599


In [5]:
from torch.utils.data import Dataset

class TTDataset(Dataset):
    def __init__(self, copa_data, tokenizer):
        self.copa_data = copa_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.copa_data)

    def __getitem__(self, idx):
        data = self.copa_data[idx]
        premise = data['premise']
        hypotheses = [data['initial'], data['counterfactual']]
        label = data['label']

        # Tokenize
        encoded_input = self.tokenizer([premise] * 2, hypotheses, padding='max_length', truncation=True, return_tensors='pt')
        encoded_input['labels'] = torch.tensor(label)

        return encoded_input


In [6]:
from transformers import RobertaTokenizer, RobertaForMultipleChoice
import torch
from torch.utils.data import DataLoader
import torch.optim as optim

# Load tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMultipleChoice.from_pretrained('roberta-base')


# Prepare the DataLoader
TT_Dataset = TTDataset(train_dataset_back, tokenizer)
dataloader = DataLoader(TT_Dataset, batch_size=16, shuffle=True)


# Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
TT_Dataset[1]

{'input_ids': tensor([[ 101, 1045, 2633,  ...,    0,    0,    0],
        [ 101, 1045, 2633,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor(0)}

In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [2]:
# Training Loop
model.train()
for epoch in range(3):  # Number of epochs
    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")


NameError: name 'model' is not defined

In [10]:
validation_dataset = TTDataset(test_dataset_back, tokenizer)
validation_dataloader = DataLoader(validation_dataset, batch_size=8)


In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.eval()
correct_predictions = 0
total_predictions = 0

accuracy, f1, precision, recall = [], [], [], []
batches = 0

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return [accuracy, f1, precision, recall]

with torch.no_grad():
    for batch in validation_dataloader:
        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']  # ground truth

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)  # predictions

        calculated_metrics = compute_metrics(labels, predictions)
        accuracy.append(calculated_metrics[0])
        f1.append(calculated_metrics[1])
        precision.append(calculated_metrics[2])
        recall.append(calculated_metrics[3])
        batches += 1

        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

overall_accuracy = correct_predictions / total_predictions
print(f"Overall Accuracy: {overall_accuracy:.4f}")

print(f"Accuracy: {sum(accuracy)/batches:.4f}")
print(f"F1: {sum(f1)/batches:.4f}")
print(f"Precision: {sum(precision)/batches:.4f}")
print(f"Recall: {sum(recall)/batches:.4f}")

print(f'Number of batches: {batches}')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and bein

Overall Accuracy: 0.6962
Accuracy: 0.6962
F1: 0.0288
Precision: 0.0800
Recall: 0.0183
Number of batches: 75


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in validation_dataloader:
        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

overall_accuracy = correct_predictions / total_predictions
print(f"Overall Accuracy: {overall_accuracy:.4f}")


Overall Accuracy: 0.6711


In [ ]:
# Specify the directory where you want to save the model
save_directory = '/content'

# Save the model to the specified directory
model.save_pretrained(save_directory)

# Optionally, save the tokenizer as well
model.config.save_pretrained(save_directory)

In [ ]:
from transformers import BertForMultipleChoice

# Specify the directory where the model is saved
load_directory = 'path/to/save/model'

# Load the model from the specified directory
loaded_model = BertForMultipleChoice.from_pretrained(load_directory)


# New Section